<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Лемматизация-текста" data-toc-modified-id="Лемматизация-текста-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Лемматизация текста</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Обучение-логистической-регрессии" data-toc-modified-id="Обучение-логистической-регрессии-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обучение логистической регрессии</a></span></li><li><span><a href="#Обучение-градиентного-бустинга" data-toc-modified-id="Обучение-градиентного-бустинга-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обучение градиентного бустинга</a></span></li><li><span><a href="#Обучение-LGBMClassifier" data-toc-modified-id="Обучение-LGBMClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обучение LGBMClassifier</a></span></li><li><span><a href="#Настройка-параметров-для-логистической-регресии" data-toc-modified-id="Настройка-параметров-для-логистической-регресии-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Настройка параметров для логистической регресии</a></span></li><li><span><a href="#Настройка-параметров-для-LGBMClassifier" data-toc-modified-id="Настройка-параметров-для-LGBMClassifier-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Настройка параметров для LGBMClassifier</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [ ]:
pip install lightgbm

In [ ]:
pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [41]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import spacy
from tqdm.notebook import tqdm
from sklearn.pipeline import Pipeline

In [3]:
RANDOM_STATE = 42

## Подготовка

In [5]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [7]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [11]:
data.drop('Unnamed: 0',axis=1, inplace = True)

Датасет toxic_comments представляет из себя таблицу состоящую из 159292 строк. Столбец text содержит текст комментария, а toxic — негативный комментарий или нет

### Лемматизация текста 

In [13]:
nlp = spacy.load('en_core_web_sm')

In [15]:
def clean_and_lemmatize(text):
    cleaned_text = re.sub(r'[^a-zA-Z ]', ' ', text).lower()
    
    doc = nlp(cleaned_text)
    
    lemmatized_tokens = [token.lemma_ for token in doc if token.is_alpha]
    
    return " ".join(lemmatized_tokens)

In [17]:
tqdm.pandas()

In [19]:
data['lemmatized_text'] = data['text'].progress_apply(clean_and_lemmatize)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [23]:
data.head()

,text,toxic,lemmatized_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man I m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


Лемматизировали текст c использованием WordNetLemmatizer
____

## Обучение

In [25]:
X = data['lemmatized_text']
y = data['toxic']

In [27]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=RANDOM_STATE, stratify=y)

In [29]:
vectorizer = TfidfVectorizer(stop_words='english')  
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

### Обучение логистической регрессии

In [31]:
model = LogisticRegression(random_state=RANDOM_STATE)

По условиям задачи, модель должна быть со значением метрики качества F1 не меньше 0.75. Немного не дотягивает. Значит будем настраивать параметры для этой модели
____

### Обучение градиентного бустинга

In [33]:
gb_model = GradientBoostingClassifier(random_state=RANDOM_STATE, verbose=1)

По условиям задачи, модель должна быть со значением метрики качества F1 не меньше 0.75. Значения далеки от необходимых, настраивать параметры не имеет смысла
_____

### Обучение LGBMClassifier

In [35]:
lgbm_model = LGBMClassifier(random_state=RANDOM_STATE, verbose=1)

По условиям задачи, модель должна быть со значением метрики качества F1 не меньше 0.75. Немного не дотягивает. Значит будем настраивать параметры для этой модели
_________

### Настройка параметров для логистической регресии

In [51]:
pipeline = Pipeline([
    ('tfidf', vectorizer),
    ('logreg', model)
])

In [53]:
param_grid = {
    'logreg__C': [0.1, 1, 10],
    #'solver': ['liblinear', 'saga']
    'logreg__max_iter': [100, 200, 300]
}

In [59]:
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='f1',
    cv=3,
    verbose=1,
    n_jobs=-1
)

In [63]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words='english')),
                                       ('logreg',
                                        LogisticRegression(random_state=42))]),
             n_jobs=-1,
             param_grid={'logreg__C': [0.1, 1, 10],
                         'logreg__max_iter': [100, 200, 300]},
             scoring='f1', verbose=1)

In [65]:
best_params = grid_search.best_params_
print("Лучшие параметры: ", best_params)

Лучшие параметры:  {'logreg__C': 10, 'logreg__max_iter': 100}


In [69]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

In [71]:
print(f"Лучшие параметры: {grid_search.best_params_}")
print(f"F1 Score: {grid_search.best_score_:.2f}")

Лучшие параметры: {'logreg__C': 10, 'logreg__max_iter': 100}
F1 Score: 0.77


По условиям задачи, модель должна быть со значением метрики качества F1 не меньше 0.75. У нас получилось 0.77 - условия выполнены

### Настройка параметров для LGBMClassifier

In [75]:
pipeline_lgbm = Pipeline([
    ('tfidf', vectorizer),
    ('lgbm', lgbm_model)
])

In [77]:
param_distributions_lgbm = {
    'lgbm__max_depth': [10, 20],
    'lgbm__learning_rate': [0.01, 0.05, 0.1],
    'lgbm__n_estimators': [100, 200, 500]
}

In [79]:
random_search_lgbm = RandomizedSearchCV(
    pipeline_lgbm, 
    param_distributions=param_distributions_lgbm, 
    n_iter=10,  
    scoring='f1', 
    cv=3, 
    verbose=1, 
    random_state=RANDOM_STATE,
    n_jobs=-1
)

In [81]:
random_search_lgbm.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[LightGBM] [Info] Number of positive: 14567, number of negative: 128795
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.556738 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 556766
[LightGBM] [Info] Number of data points in the train set: 143362, number of used features: 10456
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.101610 -> initscore=-2.179463
[LightGBM] [Info] Start training from score -2.179463


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('tfidf',
                                              TfidfVectorizer(stop_words='english')),
                                             ('lgbm',
                                              LGBMClassifier(random_state=42,
                                                             verbose=1))]),
                   n_jobs=-1,
                   param_distributions={'lgbm__learning_rate': [0.01, 0.05,
                                                                0.1],
                                        'lgbm__max_depth': [10, 20],
                                        'lgbm__n_estimators': [100, 200, 500]},
                   random_state=42, scoring='f1', verbose=1)

In [84]:
best_params_lgbm = random_search_lgbm.best_params_
print("Best parameters found: ", best_params_lgbm)

Best parameters found:  {'lgbm__n_estimators': 500, 'lgbm__max_depth': 20, 'lgbm__learning_rate': 0.05}


In [86]:
best_model_lgbm = random_search_lgbm.best_estimator_
y_pred_lgbm = best_model_lgbm.predict(X_test)

In [88]:
print(f"Лучшие параметры: {random_search_lgbm.best_params_}")
print(f"F1 Score: {random_search_lgbm.best_score_:.2f}")

Лучшие параметры: {'lgbm__n_estimators': 500, 'lgbm__max_depth': 20, 'lgbm__learning_rate': 0.05}
F1 Score: 0.77


По условиям задачи, модель должна быть со значением метрики качества F1 не меньше 0.75. У нас получилось 0.76 - условия выполнены

Для проверки модели на тестовых данных будем использовать модель логистической регресии, так как она обучается быстрее

## Выводы

In [91]:
f1 = f1_score(y_test, y_pred)
print(f"F1 Score for Logistic Regression: {f1:.2f}")

F1 Score for Logistic Regression: 0.79


В результате выполнения данного проекта были выполнены следующие шаги:
1. Данные были загружены и очищены от лишних символов и приведены их к единому регистру
2. Выполнена лемматизация текста
3. Разделили данные на обучающую и тестовую выборки
4. Использовали TF-IDF векторизацая для  преобразования текстовых данных в числовой формат
5. Обучили модели логистической регрессии, градиентного бустинга, LGBMClassifier
6. Подобрали оптимальные параметры для лучших моделей, чтобы достичь нужного значения метрики F1

По условиям задачи, модель должна быть со значением метрики качества F1 не меньше 0.75. Модель логистической регрессии на тестовых данных показала значение метрики качества F1 - 0.79. Условия выполнены


Для интернет-магазина «Викишоп» я бы рекомендовал использовать модель с наибольшим значением метрики качества F1, те модель логистической регресиии

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны